# Agent: 大模型的循环

## 一、引言

## 1. 尚未到来的涌现

> We always overestimate the change that will occur in the next two years and underestimate the change that will occur in the next ten. -- Bill Gates

去年，2024 年，是大模型产品化的一年。Sora, [GR](https://arxiv.org/abs/2402.17152), GPT-4o 等应用相继涌现。那段时间，我疯狂使用 GPT。惊叹从搜索引擎到 GPT，人类搜索效率又上了一个台阶。

当时，我隐隐感觉，若要在这一波技术变革中通过自举实现 AGI，大模型最缺的是框架能力。既然已经有了基座模型，那么通过一个柔性框架，把模型能力组织起来，相互协作，显然是当下最容易又有效的事情。

接下来的事，我们都知道了。出现了 Agent, [MCP](https://github.com/modelcontextprotocol/servers), [A2A](https://github.com/google-a2a/A2A) ...

框架能力的进步，带来了效率的提升，但是期待中的智能涌现却没有到来。一方面的原因是，当前 Agent 确实较弱，A2A 也还在发展中。但是从 Agent 当前帮助大模型提升的幅度看，要实现 AGI，恐怕还是要基座模型本身实现 AGI。借用投资领域的话说，Agent 不是 Alpha，最多算是 Smart Beta.


## 2. 小米加步枪

Agent 的处世哲学是：基座能力不够，工程方法来凑。

一个大模型的上下文窗口有限，思考时间有限。通过组织多个大模型一起工作，每个大模型负责其中一小块工作，然后像人一样，对工作进行分发、聚合、交接，形成一个工作流。如此便能突破单个大模型的限制，做更多更复杂的事情。

```mermaid
graph LR
    A[感知] --> B[规划]
    B --> C[控制]
    C -.-> D[反思]
    D -.-> E[记忆]
```

Agent 框架大多遵循如下范式：

- **感知**：一个任务过来，无论是 **被动接受** 任务信息，还是 **主动探索** 与任务有关的线索，这都属于“感知”。
- **规划**：有了对任务的认知，通过拆解任务，规划出一条或多条实现路径
- **控制**：有了实现路径，在路径上的每一个点，通过 MCP 调度资源执行任务；如果当前点遇到的问题依旧复杂，通过子 Agent 继续拆解任务
- **反思**：如果有多条实现路径，对多条路径的结果进行评估；如果只有一条实现路径，则对当前结果进行确认和校验
- **记忆**：收集本次任务信息，提炼要点，存入知识库。下次执行相似任务时，作为先验知识

一个 LLM 搞不定，就组织多个 LLMs 解决。这有点艰苦奋斗的意思。没有大炮没关系，就用小米加步枪的方式实现。

## 3. 还有 MCP

MCP 相当于引入了即时的 `Action -> React` pair。模型从此可以跟真实世界交互了。模型可以通过 MCP 连接各种服务、存储和计算资源，比如：

- **FastAPI**（可接入多种中台能力）
- **PostgreSQL**（长期记忆）
- **Redis**（存储映射关系）
- **Docker**（机器操作能力）

如果 Agent 框架是眼和脑，MCP 就是手。人类也是靠 **眼 -> 脑 -> 手** 循环，把活干起来的。

## 4. Agent 能做什么

> 回到务实的问题，Agent 能做什么？

Agent 本质是大模型的调度器，它擅长的无非是大模型擅长的那些，外加一些由组织产生的能力。

我看到有些公司，把 Agent 当作中台服务的调度器。先用 MCP 把中台能力轮询一遍，再用一个 LLM 总结轮询的结果。这当然也是一种用法，但有点画蛇添足。因为用传统的 `for loop` 和 `if else` 控制块，也能实现同样的功能。

**Agent 的价值在传统编程的能力圈之外。**

LLM 的引入增加了成本和不确定性。传统编程能实现的东西，没必要非用 Agent 实现。拿着锤子找钉子就没意思了。我们要找 Agent 能做到，但是传统编程做不到的事情。

**1）智能路由**

我们确实可以把 Agent 当调度器使用，但不能是简单的 `if else` 可以实现的那种程度。


```mermaid
graph LR
    A[任务] --> B[智能路由]
    B --> C[mcp1]
    B --> D[mcp2]
    B --> E[mcp3]
```

举个例子，我们开发一个可以 **核查客诉的 Agent**。如果用户投诉“商家未发货”，就去查商家发货记录；如果用户投诉“好几天了，我还没收到快递”，就去查物流记录；如果用户投诉“商家好没素质”，就去核查聊天记录。这样根据不同的投诉内容，查询对应的业务表的能力，就属于智能路由的能力。

**2）任务拆解**

理想情况下，我们把 MCP 开发好丢给 Agent，然后告诉它我们要什么。主 Agent 会将我们的任务拆解成若干执行步骤，每个步骤由一个子 Agent 完成。

```mermaid
graph LR
    A[任务] --> B[任务拆解]
    B --> C[步骤1]
    C --> D[步骤2]
```

举个例子，我们开发一个 **选课通知 Agent**。当我们告诉 Agent，给所有同学发选课通知，它会查询学生表，获取学生邮箱，然后调用邮箱 MCP 给同学们发送邮件；当我们要求给没选课的同学发送邮件提醒，它会查询没选课的学生，并撰写文案，发送提醒邮件；当我们要求给挂科的同学发送邮件时，它会联合成绩表，查询挂科同学的邮箱，然后发送选课通知。

**3）深入探索**

对于需要深入探索的场景，Agent 可以控制探索的深度和广度。在有了 MCP 之后，甚至拥有了“再探再报”的功能。通过前置信息和主动补全信息，动态调度计算资源。这样它可以在最小的计算量下，获取最精准的探查报告。

```mermaid
graph LR
    A[监控] -->|发现盗刷| B[风控引擎]
    B --> C[支付基础风险]
    B --> D[设备基础风险]
    B --> E[登录基础风险]
    B --> F[黑/灰名单库]

    subgraph 基础风险模块
    C[支付基础风险] --> G[风险判别]
    D --> G
    E --> G
    F --> G
    end

    G -->|研判信息不足触发| H[动态调度]

    subgraph 高计算模块
    H -.-> I[k 度风险节点] --> N[风险报告]
    H -.-> J[FPGrowth] --> N
    H -.-> K[SynchroTrap] --> N
    H -.-> L[FastUnfolding] --> N
    H -.-> M[LightGBM] --> N
    end

    N -->|黑标扩散| O[周边风险]

    subgraph 扩召模块
    O -.-> P[邻域召回路]
    O -.-> Q[Graph Embedding 召回路]
    O -.-> R[时序相似召回路]
    end

    P --> S[团伙打标]
    Q --> S
    R --> S
```

如上是一个 **风控引擎**。当监控发现一个用户在支付路由下触发风险时，监控便会向风控引擎发送一条消息，形如：

```json
{
    "uid": 1133557,
    "reason": "Card Fraud"
}
```

这时，风控引擎会触发全量基础风险排查，生成一个初步的风险报告。若当下已知风险足以判断用户是坏人，则无需进一步调用更高资源消耗的计算模块；否则结合前置信息，动态调用高计算量模块，获得完整的风险报告。此时，继续探查用户是否存在周边风险，如果存在，动态调用多路召回，实现团伙打黑。